In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!pip install transformers datasets evaluate seqeval onnx onnxruntime optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
import os
import numpy as np
import torch
import evaluate
import itertools
import pandas as pd
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments
)

# For manual ONNX export
from optimum.exporters.tasks import TasksManager
from optimum.exporters.onnx import export as onnx_export
from pathlib import Path
import shutil

def train_and_evaluate_ner(
    model_checkpoint: str,
    train_file: str,
    val_file: str,
    entity_labels: list,
    num_epochs: int,
    learning_rate: float,
    batch_size: int,
    output_dir: str,
    return_trainer: bool = False
):
    """
    Train a token classification model on a dataset and return final metrics,
    along with the Trainer (if return_trainer=True).
    """

    # Prepare labels
    all_labels = ["O"] + entity_labels
    label2id = {label: i for i, label in enumerate(all_labels)}
    id2label = {i: label for label, i in label2id.items()}

    # Load raw dataset
    data_files = {"train": train_file, "validation": val_file}
    raw_dataset = load_dataset("json", data_files=data_files)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

    # Alignment function
    def align_spans_to_tokens(examples):
        texts = examples["text"]
        batch_tokenized = tokenizer(
            texts,
            truncation=True,
            return_offsets_mapping=True
        )

        new_labels = []
        for i, offsets in enumerate(batch_tokenized["offset_mapping"]):
            spans = examples["spans"][i]
            span_entities = [(s["start"], s["end"], s["label"]) for s in spans]

            # Initialize all tokens to -100
            label_ids = [-100] * len(offsets)

            for span_start, span_end, span_label in span_entities:
                for idx, (token_start, token_end) in enumerate(offsets):
                    # If offset_mapping is (0, 0), it's likely a special token
                    if token_start == 0 and token_end == 0:
                        continue
                    if token_end > span_start and token_start < span_end:
                        label_ids[idx] = label2id[span_label]

            # Convert real tokens still -100 to "O"
            for idx, (token_start, token_end) in enumerate(offsets):
                if (token_start != 0 or token_end != 0) and label_ids[idx] == -100:
                    label_ids[idx] = label2id["O"]

            new_labels.append(label_ids)

        batch_tokenized["labels"] = new_labels
        batch_tokenized.pop("offset_mapping")
        return batch_tokenized

    # Encode dataset
    encoded_dataset = raw_dataset.map(
        align_spans_to_tokens,
        batched=True,
        remove_columns=raw_dataset["train"].column_names
    )

    train_dataset = encoded_dataset["train"]
    eval_dataset = encoded_dataset["validation"]

    # Load model
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    # Metric
    metric = evaluate.load("seqeval")

    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        true_labels = []
        true_predictions = []
        for label_ids, pred_ids in zip(labels, predictions):
            filtered_labels = []
            filtered_preds = []
            for l, p_ in zip(label_ids, pred_ids):
                if l == -100:
                    continue
                filtered_labels.append(id2label[l])
                filtered_preds.append(id2label[p_])
            true_labels.append(filtered_labels)
            true_predictions.append(filtered_preds)

        results = metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"]
        }

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",        # We'll save model each epoch, but only keep the best
        save_total_limit=1,          # Keep only the best checkpoint
        logging_strategy="steps",
        logging_steps=10,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        # disable wandb if desired
        # report_to="none",
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # -------------------------------
    # SAVE the final (best) model +
    # SAVE the tokenizer
    # -------------------------------
    trainer.save_model(output_dir)         # saves the best model
    tokenizer.save_pretrained(output_dir)  # ensure tokenizer is saved

    # Remove any leftover "checkpoint-XYZ" subfolders if they exist
    for subdir in os.listdir(output_dir):
        if subdir.startswith("checkpoint-") and os.path.isdir(os.path.join(output_dir, subdir)):
            shutil.rmtree(os.path.join(output_dir, subdir))

    # Evaluate final model
    metrics = trainer.evaluate()

    if return_trainer:
        return metrics, trainer
    else:
        return metrics




In [37]:
###########################
#    Hyperparameters      #
###########################
# model, learning rate, batch size, epochs
model_combinations = [
    ("prajjwal1/bert-tiny", 0.0001, 16, 35),
    ("prajjwal1/bert-tiny", 0.0001, 32, 35),
    ("prajjwal1/bert-tiny", 0.00005, 32, 35),
    ("microsoft/MiniLM-L12-H384-uncased", 0.0001, 16, 10),
    ("microsoft/MiniLM-L12-H384-uncased", 0.0001, 24, 10),
    ("google/mobilebert-uncased", 0.0001, 16, 8),
    ("google/mobilebert-uncased", 0.0001, 24, 8),
    ("google/mobilebert-uncased", 0.00005, 16, 8),
    ("google/mobilebert-uncased", 0.00005, 24, 8),
    ("distilbert-base-uncased", 0.0001, 16, 8),
    ("distilbert-base-uncased", 0.0001, 24, 8)
]

entity_labels = ["HOSP", "PATIENT", "STAFF", "AGE", "BIRTHDATE", "DATE",
                 "PHONE", "ID", "EMAIL", "LOC", "ORG"]
train_file = "train.json"
val_file = "validation.json"

# Make sure you set the path to a folder in your Google Drive
base_output_dir = "/content/drive/MyDrive/my_ner_experiments"

results_list = []
trainers_dict = {}

# Create the base directory if it doesn't exist
os.makedirs(base_output_dir, exist_ok=True)

##########################################################
#  Loop over each combination of checkpoint, LR, BS      #
##########################################################
for checkpoint, lr, bs, num_epochs in model_combinations:
    # Build a unique subfolder for this run
    folder_name = f"{checkpoint.replace('/', '_')}_lr{lr}_bs{bs}"
    out_dir = os.path.join(base_output_dir, folder_name)
    os.makedirs(out_dir, exist_ok=True)

    print(f"Running experiment: {checkpoint}, lr={lr}, bs={bs}")
    print(f"Saving outputs to: {out_dir}")

    # Train and get metrics + trainer
    metrics, trainer = train_and_evaluate_ner(
        model_checkpoint=checkpoint,
        train_file=train_file,
        val_file=val_file,
        entity_labels=entity_labels,
        num_epochs=num_epochs,
        learning_rate=lr,
        batch_size=bs,
        output_dir=out_dir,
        return_trainer=True
    )

    # Record final metrics
    row = {
        "model_checkpoint": checkpoint,
        "learning_rate": lr,
        "batch_size": bs,
        "precision": metrics["eval_precision"],
        "recall": metrics["eval_recall"],
        "f1": metrics["eval_f1"],
        "accuracy": metrics["eval_accuracy"],
        "output_dir": out_dir,
    }
    results_list.append(row)

    # Store the trainer for analysis
    trainers_dict[(checkpoint, lr, bs)] = trainer

    ############################################
    #   PLOT TRAINING vs VALIDATION CURVES     #
    ############################################
    log_history = trainer.state.log_history

    # 1) Training vs Validation Loss
    train_loss_vals = []
    train_steps = []
    eval_loss_vals = []
    eval_steps = []

    for entry in log_history:
        if "loss" in entry and "step" in entry:
            train_loss_vals.append(entry["loss"])
            train_steps.append(entry["step"])
        if "eval_loss" in entry and "step" in entry:
            eval_loss_vals.append(entry["eval_loss"])
            eval_steps.append(entry["step"])

    plt.figure(figsize=(8, 6))
    plt.plot(train_steps, train_loss_vals, label="Training Loss")
    plt.plot(eval_steps, eval_loss_vals, label="Validation Loss", marker='o')
    plt.xlabel("Global Step")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    loss_plot_path = os.path.join(out_dir, "loss_plot.png")
    plt.savefig(loss_plot_path)
    plt.close()

    # 2) F1 Score Over Time
    eval_f1_vals = []
    eval_f1_steps = []

    for entry in log_history:
        # 'eval_f1' typically appears in the logs as "eval_f1"
        if "eval_f1" in entry and "step" in entry:
            eval_f1_vals.append(entry["eval_f1"])
            eval_f1_steps.append(entry["step"])

    if eval_f1_vals:
        plt.figure(figsize=(8, 6))
        plt.plot(eval_f1_steps, eval_f1_vals, label="Validation F1", marker='o', color='green')
        plt.xlabel("Global Step")
        plt.ylabel("F1 Score")
        plt.title("Validation F1 Over Time")
        plt.legend()

        f1_plot_path = os.path.join(out_dir, "f1_plot.png")
        plt.savefig(f1_plot_path)
        plt.close()

    ###################################################
    #   CONVERT THE MODEL TO ONNX AND SAVE model.onnx #
    ###################################################
    print(f"Converting {checkpoint} to ONNX...")

    # 1) Reload your final (best) model from the local directory
    model_for_export = AutoModelForTokenClassification.from_pretrained(out_dir)

    # 2) Retrieve an ONNX export config
    onnx_config_constructor = TasksManager.get_exporter_config_constructor(
        model=model_for_export,
        task="token-classification",  # your task
        exporter="onnx"
    )
    onnx_config = onnx_config_constructor(model_for_export.config)

    # 3) Export to ONNX
    onnx_path = Path(os.path.join(out_dir, "model.onnx"))
    model_for_export.cpu()  # easier to export on CPU

    onnx_export(
        model=model_for_export,
        config=onnx_config,
        opset=14,          # adjust if needed
        output=onnx_path,
        device="cpu"
    )

    print(f"ONNX model saved at: {onnx_path}")
    print("-----------------------------------------------------------")


Running experiment: prajjwal1/bert-tiny, lr=0.0001, bs=16
Saving outputs to: /content/drive/MyDrive/my_ner_experiments/prajjwal1_bert-tiny_lr0.0001_bs16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-34-14257b1efbb3>:161: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.346200,1.142865,0.000000,0.000000,0.000000,0.751695
2,0.881800,0.701341,0.465046,0.352535,0.401048,0.862429
3,0.610200,0.459887,0.661504,0.688940,0.674944,0.931921
4,0.424400,0.335806,0.719828,0.769585,0.743875,0.946893
5,0.335100,0.260566,0.773420,0.817972,0.795073,0.956497
6,0.287300,0.211541,0.807522,0.841014,0.823928,0.961582
7,0.225100,0.178180,0.808696,0.857143,0.832215,0.964689
8,0.221500,0.160739,0.830022,0.866359,0.847802,0.965537
9,0.174900,0.144376,0.858407,0.894009,0.875847,0.972034
10,0.139900,0.133873,0.856512,0.894009,0.874859,0.971751


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PATIENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HOSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PATIENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HOSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.172100,0.068810,0.932870,0.928571,0.930716,0.984746
2,0.093100,0.046840,0.963218,0.965438,0.964327,0.990395
3,0.034300,0.035299,0.965753,0.974654,0.970183,0.989831
4,0.018600,0.036277,0.967963,0.974654,0.971297,0.991808
5,0.010900,0.041274,0.974654,0.974654,0.974654,0.992090
6,0.008900,0.041290,0.974654,0.974654,0.974654,0.992090
7,0.004800,0.042390,0.970183,0.974654,0.972414,0.992655
8,0.005700,0.041900,0.970183,0.974654,0.972414,0.992655


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PATIENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HOSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PATIENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HOSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

Converting distilbert-base-uncased to ONNX...
ONNX model saved at: /content/drive/MyDrive/my_ner_experiments/distilbert-base-uncased_lr0.0001_bs24/model.onnx
-----------------------------------------------------------

=== All Runs Results ===
                     model_checkpoint  learning_rate  batch_size  precision  \
4   microsoft/MiniLM-L12-H384-uncased        0.00010          24   0.972665   
3   microsoft/MiniLM-L12-H384-uncased        0.00010          16   0.979215   
7           google/mobilebert-uncased        0.00005          16   0.979167   
9             distilbert-base-uncased        0.00010          16   0.970387   
5           google/mobilebert-uncased        0.00010          16   0.974713   
10            distilbert-base-uncased        0.00010          24   0.974654   
8           google/mobilebert-uncased        0.00005          24   0.976852   
6           google/mobilebert-uncased        0.00010          24   0.972414   
0                 prajjwal1/bert-tiny        

<ipython-input-37-30d7564cca42>:172: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(1, "f1"))


In [39]:

###################################################
#  Create a DataFrame of results and display it   #
###################################################
df_results = pd.DataFrame(results_list)
df_results = df_results.sort_values("f1", ascending=False)

print("\n=== All Runs Results ===")
print(df_results)

# Best run per model
best_per_model = (
    df_results
    .groupby("model_checkpoint", group_keys=False)
    .apply(lambda group: group.nlargest(1, "f1"))
    .reset_index(drop=True)
)

print("\n=== Best Run from Each Model ===")
print(best_per_model)

# Among these best-of-each-model, pick overall best
best_overall = best_per_model.nlargest(1, "f1").iloc[0]
print("\n=== Best Overall Across All Models ===")
print(best_overall)

best_model_path = best_overall["output_dir"]
print("Best model is at:", best_model_path)

# Also, save the final results to a CSV in the base folder
results_csv_path = os.path.join(base_output_dir, "all_experiments_results.csv")
df_results.to_csv(results_csv_path, index=False)
print(f"All experiment results saved to: {results_csv_path}")


=== All Runs Results ===
                     model_checkpoint  learning_rate  batch_size  precision  \
4   microsoft/MiniLM-L12-H384-uncased        0.00010          24   0.972665   
3   microsoft/MiniLM-L12-H384-uncased        0.00010          16   0.979215   
7           google/mobilebert-uncased        0.00005          16   0.979167   
9             distilbert-base-uncased        0.00010          16   0.970387   
5           google/mobilebert-uncased        0.00010          16   0.974713   
10            distilbert-base-uncased        0.00010          24   0.974654   
8           google/mobilebert-uncased        0.00005          24   0.976852   
6           google/mobilebert-uncased        0.00010          24   0.972414   
0                 prajjwal1/bert-tiny        0.00010          16   0.909707   
1                 prajjwal1/bert-tiny        0.00010          32   0.893333   
2                 prajjwal1/bert-tiny        0.00005          32   0.812775   

      recall        f1  a

<ipython-input-39-0dc185f864b8>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(1, "f1"))


In [44]:
import os

print("\n=== ONNX Model Sizes ===")
for idx, row in df_results.iterrows():
    model_path = row["output_dir"]
    onnx_file = os.path.join(model_path, "model.onnx")

    if os.path.exists(onnx_file):
        # Calculate file size in MB
        size_bytes = os.path.getsize(onnx_file)
        size_mb = size_bytes / (1024 * 1024)

        print(
            f"Model: {row['model_checkpoint']}  |  "
            f"LR={row['learning_rate']}  |  "
            f"BS={row['batch_size']}  |  "
            f"ONNX Size: {size_mb:.2f} MB"
        )
    else:
        print(f"No ONNX file found for {row['model_checkpoint']} (output_dir={model_path}).")



=== ONNX Model Sizes ===
Model: microsoft/MiniLM-L12-H384-uncased  |  LR=0.0001  |  BS=24  |  ONNX Size: 127.00 MB
Model: microsoft/MiniLM-L12-H384-uncased  |  LR=0.0001  |  BS=16  |  ONNX Size: 127.00 MB
Model: google/mobilebert-uncased  |  LR=5e-05  |  BS=16  |  ONNX Size: 94.48 MB
Model: distilbert-base-uncased  |  LR=0.0001  |  BS=16  |  ONNX Size: 253.32 MB
Model: google/mobilebert-uncased  |  LR=0.0001  |  BS=16  |  ONNX Size: 94.48 MB
Model: distilbert-base-uncased  |  LR=0.0001  |  BS=24  |  ONNX Size: 253.32 MB
Model: google/mobilebert-uncased  |  LR=5e-05  |  BS=24  |  ONNX Size: 94.48 MB
Model: google/mobilebert-uncased  |  LR=0.0001  |  BS=24  |  ONNX Size: 94.48 MB
Model: prajjwal1/bert-tiny  |  LR=0.0001  |  BS=16  |  ONNX Size: 16.73 MB
Model: prajjwal1/bert-tiny  |  LR=0.0001  |  BS=32  |  ONNX Size: 16.73 MB
Model: prajjwal1/bert-tiny  |  LR=5e-05  |  BS=32  |  ONNX Size: 16.73 MB
